# 1 Mechine Learning

In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv("data/titanic.csv", index_col="PassengerId")
df.drop(columns=["Name","Ticket","Age","Cabin"],inplace=True)

In [6]:
# X = train test dan y = Target

X = df.drop(columns=["Survived"])
y = df["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 6), (179, 6), (712,), (179,))